<a href="https://colab.research.google.com/github/olwflynn/Learning/blob/master/PhysicsModelInference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
print(gym.__version__)

ModuleNotFoundError: No module named 'gym'

TODO (each task should be able to be completed in an hour):
- create a plot with a constant prediction - DONE
- create a naive deterministic predictor - DONE
- create trained predictor (pytorch logistic regression, nn and pyro)
- create dashboard to output error rates

In [ ]:
env = gym.make("InvertedDoublePendulum-v4", render_mode="human")
env.action_space.seed(42)


In [ ]:
# get forecast error given a predicted trajectory and observations

def forecast_error(X_obs, preds):
    
    mse = sum(np.square(X_obs - preds))
    
    return mse

In [ ]:
import numpy as np

# def plt_update(ax, colors=['b']):
#     x = np.linspace(0,1,100)
#     if ax.lines:
#         for line in ax.lines:
#             line.set_xdata(x)
#             y = np.random.random(size=(100,1))
#             line.set_ydata(y)
#     else:
#         for color in colors:
#             y = np.random.random(size=(100,1))
#             ax.plot(x, y, color)
#     fig.canvas.draw()


def live_plot(data_dict, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    # for label,data in data_dict.items():
    obs = data_dict["obs"]
    preds = data_dict["preds"]
    print("obs: ", obs, "\n preds: ",preds)

    plt.plot(obs, label="obs")
    plt.plot(range(len(obs)-1, len(obs)+len(preds)-1), preds, label="preds")
    
    plt.title(title)
    plt.grid(True)
    plt.xlabel('epoch')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

In [ ]:
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

observation, info = env.reset(seed=42)
obs_history = []

# fig,ax = plt.subplots(1,1)
# ax.set_xlabel('t')
# ax.set_ylabel('obs_idx_1')
# ax.set_xlim(0,20)
# ax.set_ylim(-5,5)

while True:
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
    obs_history.append(observation[1])
    # plt.plot(obs_history[0][0])
    # plt.show()
    preds = obs_history[-1]+[0, 0.2,0.2,0.2]
    # obs_concat_preds = obs_history
    # for pred in preds:
    #     obs_concat_preds.append(pred)
    live_plot({"obs": obs_history, "preds": preds}, title="Radians by epoch")
    time.sleep(0.5)
    if terminated or truncated:
        print(obs_history)
        print("Prediction error in last 3 steps: ", forecast_error(obs_history[-3:], preds[-3:]))
        break
        # obs_history = []
        # observation, info = env.reset()

env.close()

In [ ]:
def run_program(predictor):
    observation, info = env.reset(seed=42)
    obs_history = []

    while True:
        observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
        obs_history.append(observation[1])
        # plt.plot(obs_history[0][0])
        # plt.show()
        preds = predictor(np.array(obs_history))
        # obs_concat_preds = obs_history
        # for pred in preds:
        #     obs_concat_preds.append(pred)
        live_plot({"obs": obs_history, "preds": preds}, title="Radians by epoch")
        time.sleep(0.5)
        if terminated or truncated:
            print(obs_history)
            print("Prediction error in last 3 steps: ", forecast_error(obs_history[-3:], preds[-3:]))
            break
            # obs_history = []
            # observation, info = env.reset()

    env.close()
    

The episode ends when any of the following happens:

Truncation: The episode duration reaches 1000 timesteps.

Termination: Any of the state space values is no longer finite.

Termination: The absolutely value of the vertical angle between the pole and the cart is greater than 0.2 radian. i.e. 11.4592 degrees

1; vertical angle of the pole on the cart; -Inf; Inf; angle (rad)

*JTBD*

- Run thousands of episodes to generate training data with features (current and prev angles and other data) and labels (next few angles)
- Train forecast model to predict trajectory of angle
- Start episode with control over speed i.e. time lag between steps
- Predict expected time to end of episode at step t with measure of uncertainty
- Predict trajectory of angle in next few time steps at step t
- At end of episode output prediction errors at a few time steps


In [ ]:
# naive forecaster where we predict the angle at the next 3 steps is the same as the current angle

def naive_forecaster(X):
    # X is the all data known to the forecaster at the time. X is only the current and previous angles for now.
    
    current_angle = X[-1]
    angle_t_plus_1 = current_angle
    angle_t_plus_2 = current_angle
    angle_t_plus_3 = current_angle
    
    return np.array([current_angle, angle_t_plus_1, angle_t_plus_2, angle_t_plus_3])


In [ ]:
# naive forecaster where we predict the angle at the next 3 steps is the same as the current angle

def const_inc_forecaster(X):
    # X is the all data known to the forecaster at the time. X is only the current and previous angles for now.
    
    current_angle = X[-1]
    angle_t_plus_1 = current_angle + 0.1
    angle_t_plus_2 = current_angle + 0.2
    angle_t_plus_3 = current_angle + 0.3
    
    return np.array([current_angle, angle_t_plus_1, angle_t_plus_2, angle_t_plus_3])


In [ ]:
print(obs_history)
naive_preds = naive_forecaster(np.array(obs_history[:5]))
print(naive_preds)

In [ ]:
run_program(naive_forecaster)

NameError: name 'run_program' is not defined

In [ ]:
run_program(const_inc_forecaster)

In [ ]:
# "Learn" from single episode. NN which outputs last 3 time steps
import pandas as pd

def get_data_for_ep():
    observation, info = env.reset(seed=42)
    obs_history = []
    time_step = 0
    while True:
        observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
        obs_history.append(np.append(observation,[time_step]))
        time_step+=1
        if terminated or truncated:
                break
    env.close()
    obs_history_df = pd.DataFrame(obs_history, columns=["feature_{}".format(i) for i in range(len(obs_history[0]))])
    
    return obs_history_df



In [ ]:
print(get_data_for_ep())

In [ ]:
def create_rows_for_episode(ep_num, time_delay=0):
  env.reset(seed=42)
  for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
    time.sleep(time_delay)
    features = np.array(observation[:8]).reshape(1, 8)
    if _ == 0:
      rows_for_ep = features
    else:
      rows_for_ep = np.concatenate((rows_for_ep, features), axis=0)
    if terminated or truncated:
      labels = np.array(range(_, -1,-1)).reshape(_+1, 1)
      ep_num_arr = np.array([ep_num for i in range(len(labels))]).reshape(_+1, 1)
      assert len(labels) == len(rows_for_ep)
      rows_for_ep = np.concatenate((rows_for_ep, labels), axis=1)
      rows_for_ep = np.concatenate((rows_for_ep, ep_num_arr), axis=1)
      # print(len(rows_for_ep), type(rows_for_ep))
      return rows_for_ep

In [ ]:
# create a dataset with positions of the pendulum as features and time to terminate as label

import pandas as pd

def run_episodes(num_eps, time_delay):

    columns = []
    for i in range(8):
      columns.append(f'feature_{i}')
    columns.append('time_to_terminate')
    columns.append('ep_num')

    #create array
    num_episodes = num_eps
    for j in range(num_episodes):
      episode_rows = create_rows_for_episode(j, time_delay=time_delay)
      if j == 0:
        arr = episode_rows
      else:
        arr = np.concatenate([arr, episode_rows], axis=0)
    env.close()

    df = pd.DataFrame(arr, columns=columns)
    # print(df.head())
    # print(df.describe())
    # print(df.info())

    ep_split_df = [df[df['ep_num']==i] for i in range(1)]
    print(ep_split_df[0])
    num_features = 8
    fig, axs = plt.subplots(num_features+1, figsize=(12, 8))
    plt.suptitle('Episode plots')
    for i in range(num_features):
      for el in ep_split_df:
        ep = el['ep_num'].values[0]
        axs[i].plot(el[f'feature_{i}'], label=f'episode_{int(ep)}')
        axs[i].set_title(f'feature_{i}', fontsize='small')
    axs[8].plot(ep_split_df[0]['time_to_terminate'], label='time_to_terminate')
    axs[8].set_title('time_to_terminate', fontsize='small')
    # plt.legend(loc='right')
    plt.show()

In [ ]:
run_episodes(1, 0.5)

In [ ]:
# create model class and load models params from /models dir
from torch import nn

class DenseNet(nn.Module):
  def __init__(self):
    super(DenseNet, self).__init__()
    self.layer1 = nn.Linear(8,20)
    self.layer2 = nn.Linear(20,1)
    self.softplus = nn.Softplus()
  
  def forward(self, X):
    hidden1 = self.layer1(X)
    hidden2 = self.layer2(hidden1)

    output = self.softplus(hidden2)
    return output

PATH = 'models/10000eps_DenseNet_10000epochs.pt'

loaded_dense_net = DenseNet()
loaded_dense_net.load_state_dict(torch.load(PATH))